In [2]:
import pandas as pd
import numpy as np

In [3]:
movies = pd.read_csv('movies.csv')
credit = pd.read_csv('credits.csv')

In [4]:
movies.head(1)

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800


In [6]:
movies.shape

(4803, 20)

In [7]:
credit.shape

(4803, 4)

In [5]:
df = movies.merge(credit,on='title')

In [8]:
df.shape

(4809, 23)

In [13]:
df.columns

Index(['budget', 'genres', 'homepage', 'id', 'keywords', 'original_language',
       'original_title', 'overview', 'popularity', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'vote_average',
       'vote_count', 'movie_id', 'cast', 'crew'],
      dtype='object')

# feature selection

In [6]:
df = df[['id','title','genres','overview','production_companies','release_date','cast','crew','keywords']]

In [18]:
df['genres'][0]

'[{"id": 28, "name": "Action"}, {"id": 12, "name": "Adventure"}, {"id": 14, "name": "Fantasy"}, {"id": 878, "name": "Science Fiction"}]'

# Since the following data is in string and we need to convert it into List of dict that's why we use ast Abstract Syntex Tree

In [7]:
import ast

In [8]:
ast.literal_eval('[{"id": 28, "name": "Action"}, {"id": 12, "name": "Adventure"}, {"id": 14, "name": "Fantasy"}, {"id": 878, "name": "Science Fiction"}]')

[{'id': 28, 'name': 'Action'},
 {'id': 12, 'name': 'Adventure'},
 {'id': 14, 'name': 'Fantasy'},
 {'id': 878, 'name': 'Science Fiction'}]

In [9]:
def fetch(obj):
    y = []
    for i in ast.literal_eval(obj):
        y.append(i['name'])
    return y

In [10]:
df['genres'] = df['genres'].apply(fetch)

In [11]:
df['keywords'] = df['keywords'].apply(fetch)

In [12]:
df['production_companies']=df['production_companies'].apply(fetch)

In [13]:
def date(obj):
    y = []
    count = 0
    for i in str(obj):
        if count!=4:
            count +=1
            y.append(i)
        else:
            break
    return "".join(y)

In [16]:
date(df['release_date'][0])

'2009'

In [14]:
df['release_date']=df['release_date'].apply(date)

In [15]:
def get_cast(obj):
    y = []
    count = 0
    for i in ast.literal_eval(obj):
        if count!=3:
            count +=1
            y.append(i['name'])
        else:
            break
    return y

In [16]:
get_cast(df['cast'][0])

['Sam Worthington', 'Zoe Saldana', 'Sigourney Weaver']

In [17]:
df['cast']=df['cast'].apply(get_cast)

In [18]:
def get_director(obj):
    y = []
    for i in ast.literal_eval(obj):
        if i['job']=='Director':
            y.append(i['name'])
    return y

In [19]:
df['crew']=df['crew'].apply(get_director)

In [20]:
df['overview']=df['overview'].apply(lambda x:str(x).split())

In [21]:
df.head()

,id,title,genres,overview,production_companies,release_date,cast,crew,keywords
0,19995,Avatar,"[Action, Adventure, Fantasy, Science Fiction]","[In, the, 22nd, century,, a, paraplegic, Marin...","[Ingenious Film Partners, Twentieth Century Fo...",2009,"[Sam Worthington, Zoe Saldana, Sigourney Weaver]",[James Cameron],"[culture clash, future, space war, space colon..."
1,285,Pirates of the Caribbean: At World's End,"[Adventure, Fantasy, Action]","[Captain, Barbossa,, long, believed, to, be, d...","[Walt Disney Pictures, Jerry Bruckheimer Films...",2007,"[Johnny Depp, Orlando Bloom, Keira Knightley]",[Gore Verbinski],"[ocean, drug abuse, exotic island, east india ..."
2,206647,Spectre,"[Action, Adventure, Crime]","[A, cryptic, message, from, Bond’s, past, send...","[Columbia Pictures, Danjaq, B24]",2015,"[Daniel Craig, Christoph Waltz, Léa Seydoux]",[Sam Mendes],"[spy, based on novel, secret agent, sequel, mi..."
3,49026,The Dark Knight Rises,"[Action, Crime, Drama, Thriller]","[Following, the, death, of, District, Attorney...","[Legendary Pictures, Warner Bros., DC Entertai...",2012,"[Christian Bale, Michael Caine, Gary Oldman]",[Christopher Nolan],"[dc comics, crime fighter, terrorist, secret i..."
4,49529,John Carter,"[Action, Adventure, Science Fiction]","[John, Carter, is, a, war-weary,, former, mili...",[Walt Disney Pictures],2012,"[Taylor Kitsch, Lynn Collins, Samantha Morton]",[Andrew Stanton],"[based on novel, mars, medallion, space travel..."


In [22]:
def koma(obj):
    return [obj]

In [23]:
df['release_date']=df['release_date'].apply(koma)

In [24]:
df['tags'] = df['genres']+df['crew']+df['cast'] +df['production_companies'] + df['keywords']+df['overview'] +df['release_date']

In [25]:
df.head(1)

,id,title,genres,overview,production_companies,release_date,cast,crew,keywords,tags
0,19995,Avatar,"[Action, Adventure, Fantasy, Science Fiction]","[In, the, 22nd, century,, a, paraplegic, Marin...","[Ingenious Film Partners, Twentieth Century Fo...",[2009],"[Sam Worthington, Zoe Saldana, Sigourney Weaver]",[James Cameron],"[culture clash, future, space war, space colon...","[Action, Adventure, Fantasy, Science Fiction, ..."


In [26]:
final = df[['id','title','tags']]

In [27]:
final

,id,title,tags
0,19995,Avatar,"[Action, Adventure, Fantasy, Science Fiction, ..."
1,285,Pirates of the Caribbean: At World's End,"[Adventure, Fantasy, Action, Gore Verbinski, J..."
2,206647,Spectre,"[Action, Adventure, Crime, Sam Mendes, Daniel ..."
3,49026,The Dark Knight Rises,"[Action, Crime, Drama, Thriller, Christopher N..."
4,49529,John Carter,"[Action, Adventure, Science Fiction, Andrew St..."
...,...,...,...
4804,9367,El Mariachi,"[Action, Crime, Thriller, Robert Rodriguez, Ca..."
4805,72766,Newlyweds,"[Comedy, Romance, Edward Burns, Edward Burns, ..."
4806,231617,"Signed, Sealed, Delivered","[Comedy, Drama, Romance, TV Movie, Scott Smith..."
4807,126186,Shanghai Calling,"[Daniel Hsia, Daniel Henney, Eliza Coupe, Bill..."


In [28]:
final['tags']=final['tags'].apply(lambda x:[i.lower() for i in x])

C:\Users\HP\AppData\Local\Temp\ipykernel_22908\2087476865.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final['tags']=final['tags'].apply(lambda x:[i.lower() for i in x])


In [29]:
final['tags']=final['tags'].apply(lambda x:[ i.replace(" ","") for i in x])

C:\Users\HP\AppData\Local\Temp\ipykernel_22908\2744930535.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final['tags']=final['tags'].apply(lambda x:[ i.replace(" ","") for i in x])


In [30]:
final['tags'] = final['tags'].apply(lambda x: " ".join(x))

C:\Users\HP\AppData\Local\Temp\ipykernel_22908\2641917562.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final['tags'] = final['tags'].apply(lambda x: " ".join(x))


In [31]:
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()

In [32]:

def stem(obj):
    y = []
    for i in obj.split():
        y.append(ps.stem(i))
    return y
    

In [33]:
final['tags']=final['tags'].apply(stem)

C:\Users\HP\AppData\Local\Temp\ipykernel_22908\1765370635.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final['tags']=final['tags'].apply(stem)


In [34]:
final['tags'] = final['tags'].apply(lambda x: " ".join(x))

C:\Users\HP\AppData\Local\Temp\ipykernel_22908\2641917562.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final['tags'] = final['tags'].apply(lambda x: " ".join(x))


In [35]:
final['tags'][0]

'action adventur fantasi sciencefict jamescameron samworthington zoesaldana sigourneyweav ingeniousfilmpartn twentiethcenturyfoxfilmcorpor duneentertain lightstormentertain cultureclash futur spacewar spacecoloni societi spacetravel futurist romanc space alien tribe alienplanet cgi marin soldier battl loveaffair antiwar powerrel mindandsoul 3d in the 22nd century, a parapleg marin is dispatch to the moon pandora on a uniqu mission, but becom torn between follow order and protect an alien civilization. 2009'

In [36]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=6000,stop_words='english')

In [37]:
vector = cv.fit_transform(final['tags']).toarray()

In [38]:
vector.shape

(4809, 6000)

In [39]:
from sklearn.metrics.pairwise import cosine_similarity
similarity = cosine_similarity(vector)

In [43]:
sorted(similarity[10],reverse=True)

[1.0,
 0.4356649189097367,
 0.4125614762402809,
 0.4077862096248241,
 0.38807850084769174,
 0.3303360556061335,
 0.3024950709910101,
 0.2975595178559521,
 0.2967546048398172,
 0.28005601680560194,
 0.27185747308321606,
 0.27008488481340176,
 0.26702293491727636,
 0.2656844656620286,
 0.2643392192172763,
 0.26384545277565086,
 0.25991373003137697,
 0.2562488530553892,
 0.25583124931703105,
 0.24897004791035254,
 0.2450490147049017,
 0.24375747470904352,
 0.2435440935235262,
 0.24287224646833394,
 0.24186655996847442,
 0.24130758866434224,
 0.24093471232621327,
 0.24020125285922395,
 0.23950001573246327,
 0.23950001573246324,
 0.2392680025475199,
 0.237825747077247,
 0.23763541031440183,
 0.23686622925410047,
 0.23338001400466826,
 0.23302069121418517,
 0.23221018200641194,
 0.23195259991714115,
 0.2300894966542111,
 0.22986512849099497,
 0.22961557339788818,
 0.2293443204108173,
 0.2271060382292168,
 0.2263479546725356,
 0.22487239817113241,
 0.22467702680655838,
 0.22454435656953592,
 

In [68]:
similarity[0]

array([1.        , 0.07352146, 0.0745356 , ..., 0.04095301, 0.        ,
       0.        ])

In [59]:
def recommend(movie):
    index = final[final['title']==movie].index[0]
    distances = similarity[index]
    movies_list = sorted(list(enumerate(distances)),reverse=True,key=lambda x:x[1])[1:9]
    for i in movies_list:
        print(final.iloc[i[0]]['title'])

In [62]:
recommend('Batman Begins')

The Dark Knight
Batman
The Dark Knight Rises
Batman v Superman: Dawn of Justice
Batman
10th & Wolf
Batman & Robin
Rockaway


In [63]:
import pickle


In [64]:
pickle.dump(final.to_dict(),open('movies.pkl','wb'))
pickle.dump(similarity,open('similar.pkl','wb'))

In [65]:
final.sample(10)

,id,title,tags
237,290595,The Huntsman: Winter's War,action adventur drama cedricnicolas-troyan chr...
722,14836,Coraline,anim famili henryselick dakotafan terihatch je...
2735,54518,Justin Bieber: Never Say Never,documentari music famili jonm.chu justinbieb m...
1106,5503,The Fugitive,adventur action thriller crime mysteri andrewd...
2796,319,True Romance,action thriller crime romanc tonyscott christi...
520,38365,Grown Ups,comedi dennisdugan adamsandl salmahayek mariab...
2240,48231,A Dangerous Method,drama thriller davidcronenberg keiraknightley ...
2061,12085,Mad Money,action comedi crime calliekhouri dianekeaton q...
3935,252,Willy Wonka & the Chocolate Factory,famili fantasi melstuart genewild jackalbertso...
3359,24977,Michael Jordan to the Max,documentari jamesd.stern donkempf michaeljorda...


In [66]:
final['title']

0                                         Avatar
1       Pirates of the Caribbean: At World's End
2                                        Spectre
3                          The Dark Knight Rises
4                                    John Carter
                          ...                   
4804                                 El Mariachi
4805                                   Newlyweds
4806                   Signed, Sealed, Delivered
4807                            Shanghai Calling
4808                           My Date with Drew
Name: title, Length: 4809, dtype: object